# testing QL

In [88]:
from utils.opconNosepokeFunctions import *
from utils.supplementaryFunctions import *
from scipy.optimize import minimize
from scipy.stats import entropy
from scipy.stats import ttest_rel
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

from utils import plotSettings

# import pickle 
# with open('L:/4portProb_processed/cleandf.pkl', 'rb') as f:
#     df = pickle.load(f)

sessdf = pd.read_csv('L:/4portProb_processed/sessdf.csv')

window = 5
# sessdf = df
sessdf['rr'] = sessdf.groupby(['animal', 'session'], as_index = False).reward.rolling(window, center=True).mean().reward
bin_size = 50
# sessdf['sess_bin'] = sessdf.groupby(['animal', 'task'])['session'].transform(lambda x: pd.cut(x, bins=range(0, x.max() + bin_size, bin_size), labels=False, right=False)+1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: [Errno 2] No such file or directory: 'L:/4portProb_processed/sessdf.csv'

In [ ]:
# first folder - all data
sessdf = sessdf[~(sessdf.animal == 'Raikou')]
from utils.dfLoading import add_block_groups
blocked_df = add_block_groups(sessdf)
blocked_df = blocked_df.rename(columns = {'session':'block', 'sess_block': 'session'}).drop(columns = "Unnamed: 0")
blocked_df['datetime'] = pd.to_datetime(blocked_df['datetime'])


## week relative to each animal-task first date
blocked_df['week_task'] = blocked_df.groupby(['animal', 'task'])['datetime'].transform(
    lambda x: ((x - x.min()).dt.days // 7) + 1
)

In [85]:
lr_u = 0.033
lr_sig = 0.9128
decay = 0.1106
b = 0.
tau = 1/500
sticky = 0.8918
q0 = 0
dist_mat = abs(np.arange(4)[:, None] - np.arange(4)[None, :])

def fxn(mean, arms, permute = False):
    x = np.linspace(1, arms, arms)
    sig = 1.75/2
    amp = 0.7
    vo = 0.1
    gx = (amp*np.exp(-0.5*((x-mean)**2)/(sig**2)))+vo
    if permute:
        gx = np.random.permutation(gx)
    return gx

rounds = 100
# l = [fxn(np.random.randint(1, 5), 4, True) for _ in range(10000)]

p = np.zeros(shape = (rounds, 100, 4))
q = np.zeros(shape = (rounds, 100, 4))
a = np.zeros(shape = (rounds, 100), dtype = int)
r = np.zeros(shape = (rounds, 100), dtype = int)
for rnd in range(rounds):
    q[rnd, 0, :] = np.ones(4)*q0
    rp = l[rnd]
    print (rp)
    upper = 0
    n_a = np.ones(4)
    prev_a = np.zeros(4)

    for t in range(100):
        # decay q-val
        # print ('q value', q[rnd, t, :])
        q[rnd, t, :] = q[rnd, t, :] - (decay*q[rnd, t, :])
        # print('decayed q ',q[rnd, t, :])

        # softmax and choose
        q_upper = q[rnd, t, :] + b*upper
        p[rnd, t, :] = q_upper/tau + prev_a*sticky
        # print('q+ucb*inv temp + sticky', p[rnd, t, :])
        p[rnd, t, :] = p[rnd, t, :] - np.max(p[rnd, t, :])
        # print('p pre-softmax', p[rnd, t, :])
        p[rnd, t, :] = np.exp(p[rnd, t, :])
        p[rnd, t, :] = p[rnd, t, :]/np.sum(p[rnd, t, :])
        # print('p value ',p[rnd, t, :])
        coin = np.random.multinomial(1, p[rnd, t, :])
        a[rnd, t] = int(np.arange(4)[coin.nonzero()[0][0]])
        # print('a ', a[rnd, t])
        # increment n 
        n_a[a[rnd, t]] += 1

        # reward
        r[rnd, t] = np.random.choice(2, p = [1-rp[a[rnd, t]], rp[a[rnd, t]]])
        # print('r', r[rnd, t])
        if t<99:
            # determine if locality is involved
            if lr_sig > 0:
                lr = lr_u * np.exp(- (dist_mat[a[rnd, t], :]**2) / (2 * (lr_sig**2)))
                # print( 'lr', lr)
                q[rnd, t+1, :] = q[rnd, t, :] + lr*(r[rnd, t] - q[rnd, t, :])
                # print('current q-val', q[rnd, t, :])
                # print('new q-val', q[rnd, t+1, :])
            else:
                # update q-val
                q[rnd, t+1, a[rnd, t]] = q[rnd, t, a[rnd, t]] + lr*(r[rnd, t] - q[rnd, t, a[rnd, t]])
            
            q[rnd, t+1, :] = np.clip(q[rnd, t+1, :], 0, 1)
            # print('clipped q-val', q[rnd, t+1, :])
            upper = np.sqrt(2*np.log(t+1))/n_a
            # print(upper)

            # calc sticky param
            prev_a = np.zeros(4)
            prev_a[a[rnd, t]] += 1
            # print('prev-a', prev_a)

[0.15135876 0.8        0.10196115 0.46431508]
[0.8        0.46431508 0.46431508 0.15135876]
[0.15135876 0.10196115 0.46431508 0.8       ]
[0.46431508 0.8        0.46431508 0.15135876]
[0.15135876 0.10196115 0.8        0.46431508]
[0.46431508 0.8        0.15135876 0.46431508]
[0.15135876 0.46431508 0.46431508 0.8       ]
[0.46431508 0.8        0.10196115 0.15135876]
[0.46431508 0.8        0.15135876 0.46431508]
[0.15135876 0.46431508 0.46431508 0.8       ]
[0.46431508 0.8        0.46431508 0.15135876]
[0.46431508 0.10196115 0.15135876 0.8       ]
[0.46431508 0.46431508 0.8        0.15135876]
[0.10196115 0.46431508 0.15135876 0.8       ]
[0.10196115 0.8        0.46431508 0.15135876]
[0.8        0.10196115 0.46431508 0.15135876]
[0.46431508 0.15135876 0.10196115 0.8       ]
[0.15135876 0.46431508 0.46431508 0.8       ]
[0.8        0.46431508 0.15135876 0.10196115]
[0.8        0.15135876 0.46431508 0.46431508]
[0.46431508 0.46431508 0.8        0.15135876]
[0.8        0.46431508 0.10196115 

In [ ]:
def nLL_ql_gen(x0, df, arms):
    beta, tau, lr_u, lr_sig, decay, sticky, q0 = x0 
    sessions = df.session.value_counts()
    chosen_action = df.port.to_numpy().reshape(-1, 100)
    rewarded = df.reward.to_numpy().reshape(-1, 100)
    P = np.ones_like(rewarded)*1e-7
    session_end = np.cumsum(sessions)
    ll = 0
    rounds = chosen_action.shape[0]
    trials = chosen_action.shape[1]

    dist_mat = abs(np.arange(arms)[:, None] - np.arange(arms)[None, :])
    
    q = np.zeros(shape = (rounds, trials, arms))

    for rnd in range(rounds):
        q[rnd, 0, :] = np.ones(arms)*q0
        upper = 0
        n_a = np.ones(arms)
        prev_a = np.zeros(arms)

        for t in range(1,trials):
            # decay q-val
            q[rnd, t, :] = q[rnd, t, :] - (decay*q[rnd, t, :])

            # chosen_action
            a = int(chosen_action[rnd, t]-1)

            # increment n 
            n_a[a] += 1

            # rewarded?
            r = rewarded[rnd, t]
            
            # determine if locality is involved
            if lr_sig > 0:
                lr = lr_u * np.exp(-(dist_mat[a, :]**2) / (2 * (lr_sig**2)))
                q[rnd, t, :] = q[rnd, t-1, :] + lr*(r - q[rnd, t-1, :])

            else:
                # update q-val
                q[rnd, t, a] = q[rnd, t-1, a] + lr*(r - q[rnd, t-1, a])
            
            q[rnd, t, :] = np.clip(q[rnd, t, :], 0, 1)
            upper = np.sqrt(2*np.log(t+1))/n_a

            # calc sticky param
            prev_a = np.zeros(arms)
            prev_a[int(chosen_action[rnd, t-1]-1)] += 1
            
            # calculate softmax
            q_upper = q[rnd, t, :] + beta*upper
            p = q_upper/tau + prev_a*sticky
            p = p - np.max(p)
            p = np.exp(p)/np.sum(np.exp(p))
            P[rnd, t] = p[a]

    P[P == 1e-7] = np.nan
    ll += np.nansum(np.log(P))
    nll = -ll
        
    return nll, q, chosen_action

# testing different GP packages


## load train/test data

In [ ]:
## load all data, divide into test-train
# load data - first block only and separate into test-train 20-80
def load_data_into_dict(seed_n = 42):
    import pickle 
    with open('S:/fileTransferFromWindows/cleandf.pkl', 'rb') as f:
        df = pickle.load(f)

    # subset for expert performance
    subset_df = df[(df.sess_bin>=4) & (df.task == 'unstr')]
    from utils.dfLoading import subset_trials, add_block_groups
    
    # add block groups and select only first block 
    subset_df = add_block_groups(subset_df)
    # subset_df = subset_df[subset_df.block_group==1]
    trialsinsess = 100

    # then subset for first 100 trials
    subset_df = subset_trials(subset_df, trialsinsess=trialsinsess, head_trials = trialsinsess)

    np.random.seed(seed_n)

    test_sess_dict = {}
    # pick 10% sessions for testing
    for animal in subset_df.animal.unique():
        unique_sess = subset_df[subset_df.animal == animal].session.nunique()
        test_sess = np.random.choice(subset_df[subset_df.animal == animal].session.unique(), int(unique_sess*0.2), replace = False)
        test_sess_dict[animal] = test_sess
        subset_df.drop(subset_df[(subset_df.animal == animal) & (subset_df.session.isin(test_sess))].index, inplace = True)

    sess_dict = {key: group for key, group in subset_df.groupby('animal')}

    print('loaded data')
    return sess_dict, test_sess_dict

## sklearn

In [13]:
## sklearn
import numpy as np
from sklearn.gaussian_process import GaussianProcessClassifier     # generate GP 
from sklearn.gaussian_process.kernels import RBF, Product, ConstantKernel
from scipy.special import expit
from scipy.optimize import minimize
# train_data, test_data = load_data_into_dict()

def nLL_gp_ucb_sklearn(x0, df, arms):
    beta, tau, ls = x0
    ll = 0
    sessions = df.session.value_counts()
    chosen_action = df.port.to_numpy()
    # chosen_action = np.concatenate([[1000, 1000], chosen_action])
    rewarded = df.reward.to_numpy()
    # rewarded = np.concatenate([[0, 1], rewarded])
    p = np.ones(len(df))*1e-7
    session_ends = np.cumsum(sessions)
    
    sess_num = 0
    sess_start = True

    # gp settings
    X = np.linspace(1,arms,arms).reshape(arms,1)
    knl = Product(RBF(length_scale=ls), ConstantKernel(constant_value=1**2))

    for trial in range(len(chosen_action)):
        # check if sess restarted
        if sess_start == True:
            # if group.block_group.unique()[0] == 1:
            gp = GaussianProcessClassifier(kernel=knl, optimizer = None)
            start_trial = trial

        sess_start = False
        
        # if len(np.unique(rewarded[start_trial:trial])) > 1:
        if trial - start_trial>1:
            # what actions were taken so far
            a = np.concatenate([[1000, 1000], chosen_action[start_trial:trial]])

            # what rewards were given for each action
            r = np.concatenate([[0, 1],rewarded[start_trial:trial]])
            # print(a, r)
            # update gp using all the info we got so far on actions and rew
            gp.fit(a.reshape(-1, 1), r)

            # get latent mean and variance
            mu, var = gp.latent_mean_and_variance(X)
            mu_star = expit(mu)
            sd_star = expit(mu+np.sqrt(var)) - expit(mu)

            # calculate probability of taking any action
            input_to_softmax = (mu_star + beta*sd_star)/tau
            P = np.exp(input_to_softmax - max(input_to_softmax))
            P = P/ np.sum(P)
            P = np.clip(P, a_min = 1e-7, a_max = None)

            # what was probability of action taken?
            chosen = int(chosen_action[trial]-1)
            p[trial] = P[chosen]

        # check how many trials elapsed
        if trial == session_ends.iloc[sess_num]:
            sess_start=True
            sess_num += 1
    p[p == 1e-7] = np.nan
    ll += np.nansum(np.log(p))
    nll = -ll
    return nll


In [14]:
nLL_gp_ucb_sklearn([0.1, 1, 1], train_data['test05022023'], 4)

np.float64(7140.387636868272)

In [18]:
opt_sklearn.x

array([-27.34294163,   1.03625047,   0.61151958])

In [16]:
opt_sklearn = minimize(nLL_gp_ucb_sklearn, [0.1, 1, 1], args = (train_data['test05022023'], 4), options = {'disp':True})

In [19]:
opt_sklearn2 = minimize(nLL_gp_ucb_sklearn, [0.1, 1, 1], args = (train_data['test05022023'], 4), options = {'disp':True}, method = 'L-BFGS-B', bounds = [(0, 500), (0.0015, 100), (1e-2, 1e+2)])


In [29]:
from pybads import BADS

def fun_for_pybads(x):
    return nLL_gp_ucb_sklearn(x, train_data['test05022023'], 4)
bads = BADS(fun_for_pybads, None, [0, 0.0015, 1e-2], [10, 100, 1e+2])
opt_sklearn_bads = bads.optimize()

bads:TooCloseBounds: For each variable, hard and plausible bounds should not be too close. Moving plausible bounds.
Initial starting point is invalid or not provided. Initial point randomly sampled uniformly from plausible box

Variables (index) internally transformed to log coordinates: [[0 1]
 [0 2]]
Beginning optimization of a DETERMINISTIC objective function

 Iteration    f-count         f(x)           MeshScale          Method             Actions
     0           2         7473.64               1                                 Uncertainty test
     0           6         7472.46               1         Initial mesh            Initial points
     0           9         7472.26             0.5         Refine grid             Train
     1          11         7472.13             0.5     Incremental search (ES-ell)        
     1          12         7472.11             0.5     Incremental search (ES-wcm)        
     1          13         7470.05             0.5     Successful search (

In [28]:
opt_sklearn_bads.x

array([4.71236328e+01, 1.50206880e-03, 1.00312876e-02])

In [21]:
opt_sklearn2.x

array([0.        , 0.00150312, 0.01      ])

## GPJax (env not set up)

In [2]:
## gpjax 

# from utils.supplementaryFunctions import tic, toc
import jax
from flax import nnx

from jax import config
import jax.random as jr
import jax.numpy as jnp
import jax.scipy as jsp
import optax as ox
import numpy as np 
import numpyro.distributions as npd

config.update("jax_enable_x64", True) ## Enable 64-bit float precision for jax. 

import gpjax as gpx
from gpjax.linalg import PSD, lower_cholesky, solve
from gpjax.parameters import Parameter

# import dill as pickle 
from pybads import BADS
import pandas as pd
import numpy as np


def _setup_GP(num_datatpoints, hyperparams):
    if 'lengthscale' in hyperparams.keys():
        kernel = gpx.kernels.RBF(lengthscale = hyperparams['lengthscale'])
    else:
        kernel = gpx.kernels.RBF()
    meanf = gpx.mean_functions.Constant()
    prior = gpx.gps.Prior(mean_function = meanf, kernel = kernel)
    likelihood = gpx.likelihoods.Bernoulli(num_datapoints = num_datatpoints)
    posterior = prior * likelihood
    return likelihood, posterior

def _compute_laplace(posterior, X_train, X_test, D, jitter = 1e-6):
    """
    Modified from here, 
    
    """
    # Compute the latent function value map at each of the training points. 
    # Kxx = posterior.prior.kernel.gram(X_train)
    # Kxx += jnp.eye(D.n) * jitter 
    # Kxx = PSD(Kxx) # Forces positive-semi definite 
    # Lx = lower_cholesky(Kxx)
    # f_hat = Lx @ posterior.latent.value # Compute the mean of the posterior distribution. 

    # Now we compute the negative Hessian, using autodifferentiation. 
    graphdef, params, *static_state = nnx.split(posterior, Parameter, ...)

    def loss(params, D):
        model = nnx.merge(graphdef, params, *static_state)
        return -gpx.objectives.log_posterior_density(model, D)
    
    jacobian = jax.jacfwd(jax.jacrev(loss))(params, D)
    H = jacobian["latent"].value["latent"].value[:, 0, :, 0]
    L = jnp.linalg.cholesky(H + jnp.eye(D.n) * jitter)

    # Compute H-1 
    L_inv = jsp.linalg.solve_triangular(L, jnp.eye(D.n), lower = True)
    H_inv = jsp.linalg.solve_triangular(L.T, L_inv, lower = False)
    ## Do not need these, do not compute... 
    # LH = jnp.linalg.cholesky(H_inv)
    # laplace_approximation = npd.MultivariateNormal(f_hat.squeeze(), scale_tril = LH)

    # Construct laplace distribution on test values. 
    map_latent_dist = posterior.predict(X_test, train_data = D)

    Kxt = posterior.prior.kernel.cross_covariance(X_train, X_test)
    Kxx = posterior.prior.kernel.gram(X_train)
    Kxx += jnp.eye(D.n) * jitter 
    Kxx = PSD(Kxx)

    Kxx_inv_Kxt = solve(Kxx, Kxt)
    laplace_cov_term = jnp.matmul(jnp.matmul(Kxx_inv_Kxt.T, H_inv), Kxx_inv_Kxt)

    mean = map_latent_dist.mean
    covariance = map_latent_dist.covariance_matrix + laplace_cov_term
    L = jnp.linalg.cholesky(covariance)
    laplace_latent_dist = npd.MultivariateNormal(jnp.atleast_1d(mean.squeeze()), scale_tril = L)

    return laplace_latent_dist

def _compute_softmax(logits, beta):
    """
    Compute softmax. 

    Parameters 
    ---------- 
    logits : 
    beta : temperature of the softmax

    Returns 
    ----------
    values : probabilities at each point after applying the softmax. 
    """
    return jax.nn.softmax(logits / beta) 

def _compute_ucb_softmax_sticky(mu, std, prev_choice, ucb, beta, stickyness):
    """ 
    Compute UCB, add a sticky term and then apply softmax to convert into probabilities. 

    Parameters
    ----------
    mu, std, prev_choice, ucb, beta, stickyness
    
    Returns
    ---------- 
    probabilities
    """
    
    prob = (mu + ucb * std)*beta + prev_choice*stickyness 
    prob = prob - jnp.max(prob) # To deal with overflow errors apparently? 
    return jax.nn.softmax(prob)

def _compute_ucb_softmax(mu, std, ucb, beta):
    """
    Compute UCB and apply softmax to convert into probabilities.
    Parameters
    ----------
    mu, std, ucb, beta

    Returns
    ---------- 
    probabilities
    """
    prob = (mu + ucb * std)*beta
    prob = prob - jnp.max(prob) # To deal with overflow errors apparently? 
    return jax.nn.softmax(prob)

def run_GP_single_trial(actions, rewards, hyperparams, key = jr.PRNGKey(42), jitter = 1e-6):

    assert actions.shape == rewards.shape 

    ## Remove NaN trials and create dataset to work with. 
    mask = ~np.isnan(actions)
    actions = np.expand_dims(actions[mask], 1)
    rewards = np.expand_dims(rewards[mask], 1)    
    D = gpx.Dataset(X = actions, y = rewards)
    action_space = np.expand_dims(np.arange(1,5), 1)

    _, posterior = _setup_GP(D.n, hyperparams)
    print("setup GP.")

    ## Fit GP using Adam, get first approximation of MAP estimate 
    opt_posterior, _ = gpx.fit(
        model = posterior,
        objective = lambda p, d: -gpx.objectives.log_posterior_density(p, d),
        train_data = D,
        optim = ox.adamw(learning_rate=0.01),
        num_iters=100,
        key = key,
        trainable=Parameter, 
        verbose = False,
    )
    print("optimized GP.")

    # If using softmax, uncomment these lines, do not need the Laplace approximation. 
    # map_latent_dist = opt_posterior.predict(angle_space, train_data = D)
    # predictive_dist = opt_posterior.likelihood(map_latent_dist)
    # predictive_mean = predictive_dist.mean
    # probabilities = _compute_softmax(predictive_mean, hyperparams['beta'])

    # Now compute Laplace approximation, and evaluate it on range, Do not need the Laplace approximation imo.
    laplace_dist = _compute_laplace(opt_posterior, actions, action_space, D)
    print("computed laplace.")
    predictive_dist = opt_posterior.likelihood(laplace_dist)
    predictive_mean = predictive_dist.mean
    predictive_std = jnp.sqrt(predictive_dist.variance)


    prev_choices = np.zeros(shape = action_space.shape[0])
    idx = (action_space.flatten() == actions[-1]) # find index for last angle
    prev_choices[idx] = 1 
    probabilities = _compute_ucb_softmax(predictive_mean, predictive_std, hyperparams['ucb'], hyperparams['beta'])
    print("computed probabilities.")
    return probabilities

def fit_GP_animal(actions, rewards, hyperparams):

    nsessions, ntrials = actions.shape

    action_space = np.arange(1,5)   #[1,2,3,4]
    ll = np.zeros(shape = (nsessions, ntrials)) * np.nan 

    ## Parallelize this, otherwise its going to be insanely slow.     
    for s in range(nsessions):
        for t in range(1, ntrials):

            if t == (ntrials - 1):
                break 
            # Fit GP using trials upto t, and then evaluate probability of choosing angle t+1 
            curr_actions = actions[s, :t]
            curr_rewards = rewards[s, :t]

            # Fit GP and get probabilities. 
            probabilities = run_GP_single_trial(curr_actions, curr_rewards, hyperparams)
            log_probability = np.log(probabilities)
            # idx = np.argwhere(np.round(actions[s, t], 2) == action_space)
            idx = (action_space.flatten() == actions[s, t])
            ll[s, t] = float(log_probability[idx])
            print(f"fit {t} trials.")
            
    return ll

def optimize_gp(dataset, rat): #(rat, datadir)

    # bd = BehaviouralData()
    actions = dataset[rat].port.to_numpy().reshape(-1, 100)[0:100]
    rewards = dataset[rat].reward.to_numpy().reshape(-1, 100)[0:100]

    def likelihood(x, actions = actions, rewards = rewards):

        lengthscale, beta, ucb, stickyness = x
        hyperparams = {'beta': beta, 
                       'ucb': ucb,
                       'lengthscale': lengthscale,
                        'stickyness':stickyness}
        
        ll = fit_GP_animal(actions, rewards, hyperparams)
        return -np.nansum(ll)
    
    lb = np.array([0, 0, 0, 0])
    ub = np.array([10, 10, 10, 1e-5])
    plb = np.array([0.1, 0.1, 0.1, 0])
    pub = np.array([1, 1, 1, 1e-5])
    bads = BADS(likelihood, None, lb, ub, plb, pub, options = {'random_seed':42})
    optresult = bads.optimize()
    # with open(f'{datadir}/{rat+1}-gp.pkl', 'wb') as f:
    #     pickle.dump(optresult, f)
    return optresult


# testing how long it takes for 1 session of 100 trials
# train_data, test_data = load_data_into_dict()
rat = 'test05022023'
actions = train_data[rat].port.to_numpy().reshape(-1, 100)[:1, 0:100]
rewards = train_data[rat].reward.to_numpy().reshape(-1, 100)[:1, 0:100]
def likelihood(x, actions = actions, rewards = rewards):

    lengthscale, beta, ucb, stickyness = x
    hyperparams = {'beta': beta, 
                    'ucb': ucb,
                    'lengthscale': lengthscale,
                    'stickyness':stickyness}
    
    ll = fit_GP_animal(actions, rewards, hyperparams)
    return -np.nansum(ll)

x = [0.1, 0.1, 0.1, 0.1]
nll = likelihood(x, actions, rewards)
print(nll)

# optresult = optimize_gp(train_data, rat)
# print(optresult)

ModuleNotFoundError: No module named 'jax'

## GPy

In [2]:
## GPy
import GPy
import numpy as np
from scipy.special import expit
from scipy.optimize import minimize
train_data, test_data = load_data_into_dict()

def nLL_gp_ucb_gpy(x0, df, arms):
    beta, tau, ls, lr_sig = x0
    ll = 0
    sessions = df.session.value_counts()
    chosen_action = df.port.to_numpy()
    rewarded = df.reward.to_numpy()
    p = np.ones(len(df))*1e-7
    session_ends = np.cumsum(sessions)
    
    sess_num = 0
    sess_start = True
    # lik = GPy.likelihoods.Bernoulli()

    # gp settings
    X = np.linspace(1,arms,arms).reshape(arms,1)
    knl = GPy.kern.RBF(input_dim = 1, variance=lr_sig, lengthscale=ls)

    for trial in range(len(chosen_action)):
        # check if sess restarted
        if sess_start == True:
            # if group.block_group.unique()[0] == 1:
            start_trial = trial

        sess_start = False
        
        if len(np.unique(rewarded[start_trial:trial])) > 1:
            # what actions were taken so far
            a = chosen_action[start_trial:trial]

            # what rewards were given for each action
            r = rewarded[start_trial:trial]

            # update gp using all the info we got so far on actions and rew
            # try:
            gp = GPy.models.GPClassification(a.reshape(-1, 1), r.reshape(-1, 1), kernel = knl, inference_method = GPy.inference.latent_function_inference.expectation_propagation.EP())
            # except Exception as e:
            #     print(a, r, sess_num, trial)
            #     print(e)
            #     break
            mu, var = gp.predict_noiseless(X, kern = knl)
            # print(mu, var)
            mu_star = expit(mu)
            sd_star = expit(mu+np.sqrt(var)) - expit(mu)

            # calculate probability of taking any action
            input_to_softmax = (mu_star + beta*sd_star)/tau
            P = np.exp(input_to_softmax - max(input_to_softmax))
            P = P/ np.sum(P)
            P = np.clip(P, a_min = 1e-7, a_max = None)
    
            # what was probability of action taken?
            chosen = int(chosen_action[trial]-1)
            p[trial] = P[chosen]

        # check how many trials elapsed
        if trial == session_ends.iloc[sess_num]:
            sess_start=True
            sess_num += 1
    p[p == 1e-7] = np.nan
    ll += np.nansum(np.log(p))
    nll = -ll
    return nll

 c:\Users\dlab\rishika_sim\utils\dfLoading.py:60: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 c:\Users\dlab\rishika_sim\utils\dfLoading.py:63: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 c:\Users\dlab\rishika_sim\utils\dfLoading.py:65: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

loaded data


In [4]:
nLL_gp_ucb_gpy([0.1, 1, 1, 2], train_data['test05022023'], 4)

7005.031712683343

In [8]:
opt_gpy = minimize(nLL_gp_ucb_gpy, [0.1, 1, 1, 2], args = (train_data['test05022023'], 4))

KeyboardInterrupt: 

In [12]:
knl = GPy.kern.RBF(input_dim = 1, variance=1, lengthscale=1)
actions = train_data['test05022023'].port.to_numpy().reshape(-1, 100)
rewards = train_data['test05022023'].reward.to_numpy().reshape(-1, 100)
for trial in range(1, 101):
    gp = GPy.models.GPClassification(actions[0, :trial].reshape(-1, 1), rewards[0, :trial].reshape(-1, 1), kernel = knl, inference_method = GPy.inference.latent_function_inference.expectation_propagation.EP())
    gp.optimize()
    mu, var = gp.predict_noiseless(np.array([1,2,3,4]).reshape(-1, 1))
    print(mu, var)
    # print(gp)

[[-0.07635476]
 [-0.34219828]
 [-0.56418958]
 [-0.34219828]] [[0.99416995]
 [0.88290034]
 [0.68169011]
 [0.88290034]]
[[-0.16778838]
 [-0.75197537]
 [-1.23979778]
 [-0.75197537]] [[2.32067894]
 [1.7833705 ]
 [0.81174897]
 [1.7833705 ]]
[[-1.94384472]
 [-1.94389576]
 [-1.94388582]
 [-1.94381489]] [[0.93576123]
 [0.9355847 ]
 [0.93564517]
 [0.93594265]]
[[-2.58604409]
 [-2.58612522]
 [-2.58612521]
 [-2.58604407]] [[1.52563956]
 [1.52522005]
 [1.52522006]
 [1.52563957]]
[[-6.09212810e-06]
 [-6.09242191e-06]
 [-6.09252458e-06]
 [-6.09243611e-06]] [[2.7938513e-06]
 [2.7938513e-06]
 [2.7938513e-06]
 [2.7938513e-06]]
[[-8.91609086e-06]
 [-8.91651047e-06]
 [-8.91665035e-06]
 [-8.91651047e-06]] [[2.79383992e-06]
 [2.79383992e-06]
 [2.79383992e-06]
 [2.79383992e-06]]
[[-0.64451207]
 [-0.64457212]
 [-0.64461195]
 [-0.64463155]] [[0.16160254]
 [0.16157833]
 [0.16158019]
 [0.16160812]]
[[-0.93792732]
 [-0.93805053]
 [-0.93814431]
 [-0.93820865]] [[0.21469995]
 [0.21465229]
 [0.21465984]
 [0.2147226

## GPflow

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import gpflow
from scipy.special import expit
from scipy.optimize import minimize
train_data, test_data = load_data_into_dict()

actions = train_data['test05022023'].port.to_numpy().reshape(-1, 100)
rewards = train_data['test05022023'].reward.to_numpy().reshape(-1, 100)

c:\Users\dlab\rishika_sim\utils\dfLoading.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = 0
c:\Users\dlab\rishika_sim\utils\dfLoading.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = df.groupby(['animal', 'task']).sess_block.cumsum()
c:\Users\dlab\rishika_sim\utils\dfLoading.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

loaded data


In [7]:
## GPflow

def nLL_gp_ucb_gpflow(x0, df, arms):
    beta, tau, ls, lr_sig = x0
    ll = 0
    sessions = df.session.value_counts()
    chosen_action = df.port.to_numpy()
    rewarded = df.reward.to_numpy()
    p = np.ones(len(df))*1e-7
    session_ends = np.cumsum(sessions)
    
    sess_num = 0
    sess_start = True
    lik = gpflow.likelihoods.Bernoulli()

    # gp settings
    X = np.linspace(1,arms,arms).reshape(arms,1)
    knl = gpflow.kernels.SquaredExponential(variance=lr_sig, lengthscales=ls)

    for trial in range(len(chosen_action)):
        # check if sess restarted
        if sess_start == True:
            # if group.block_group.unique()[0] == 1:
            start_trial = trial

        sess_start = False
        
        if len(np.unique(rewarded[start_trial:trial])) > 1:
        # what actions were taken so far
            a = chosen_action[start_trial:trial]

            # what rewards were given for each action
            r = rewarded[start_trial:trial]

            # update gp using all the info we got so far on actions and rew
            gp = gpflow.models.VGP(
                (a.reshape(-1, 1), r.reshape(-1, 1)),
                kernel = knl,
                likelihood = lik
            )
            opt = gpflow.optimizers.Scipy()
            opt.minimize(gp.training_loss, gp.trainable_variables)
            
            mu, var = gp.predict_f(X)
            # print(mu, var)
            mu_star = expit(mu)
            sd_star = expit(mu+np.sqrt(var)) - expit(mu)

            # calculate probability of taking any action
            input_to_softmax = (mu_star + beta*sd_star)/tau
            P = np.exp(input_to_softmax - max(input_to_softmax))
            P = P/ np.sum(P)
            P = np.clip(P, a_min = 1e-7, a_max = None)

            # what was probability of action taken?
            chosen = int(chosen_action[trial]-1)
            p[trial] = P[chosen]

        # check how many trials elapsed
        if trial == session_ends.iloc[sess_num]:
            sess_start=True
            sess_num += 1
    p[p == 1e-7] = np.nan
    ll += np.nansum(np.log(p))
    nll = -ll
    return nll

In [8]:
nLL_gp_ucb_gpflow([0.1, 1, 1, 2], train_data['test05022023'], 4)

C:\Users\dlab\AppData\Local\Temp\ipykernel_20488\769113110.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p[trial] = P[chosen]


7347.770040366276

In [4]:
# for sess in range(actions.shape[0]):
for trial in range(2,101):
    Y = rewards[0, :trial].reshape(-1, 1)
    X = actions[0, :trial].reshape(-1, 1)
    mdl = gpflow.models.VGP(
        (X, Y),
        kernel = gpflow.kernels.SquaredExponential(),
        likelihood = gpflow.likelihoods.Bernoulli()
    )
    opt = gpflow.optimizers.Scipy()
    opt.minimize(mdl.training_loss, mdl.trainable_variables)
    mu, var = mdl.predict_f(np.array([1,2,3,4], dtype = float).reshape(-1, 1))
    print(mu, f"mean trial {trial}")
    print(var, f"var trial {trial}")
    # print(sess)

tf.Tensor(
[[-0.44435427]
 [-1.99145769]
 [-3.28335865]
 [-1.99145769]], shape=(4, 1), dtype=float64) mean trial 2
tf.Tensor(
[[14.19826143]
 [10.42980172]
 [ 3.61524928]
 [10.42980172]], shape=(4, 1), dtype=float64) var trial 2
tf.Tensor(
[[-4.42414868]
 [-4.42414869]
 [-4.42414869]
 [-4.42414867]], shape=(4, 1), dtype=float64) mean trial 3
tf.Tensor(
[[4.81339482]
 [4.81339472]
 [4.81339474]
 [4.8133949 ]], shape=(4, 1), dtype=float64) var trial 3
tf.Tensor(
[[-5.20576098]
 [-5.20576099]
 [-5.20576099]
 [-5.20576098]], shape=(4, 1), dtype=float64) mean trial 4
tf.Tensor(
[[5.64754031]
 [5.64754013]
 [5.64754009]
 [5.64754021]], shape=(4, 1), dtype=float64) var trial 4
tf.Tensor(
[[-1.56619414e-04]
 [-1.18306967e-01]
 [-9.98148921e-01]
 [-1.18311757e-01]], shape=(4, 1), dtype=float64) mean trial 5
tf.Tensor(
[[0.44063165]
 [0.93490465]
 [0.46081184]
 [0.94204368]], shape=(4, 1), dtype=float64) var trial 5
tf.Tensor(
[[-1.71860880e-04]
 [-1.91320878e-01]
 [-1.44924977e+00]
 [-1.9135634

## GPyTorch

In [ ]:
## GPyTorch

## block-wise fitting for all data

In [40]:
block_train_data, block_test_data = load_data_into_dict()

c:\Users\dlab\rishika_sim\utils\dfLoading.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = 0
c:\Users\dlab\rishika_sim\utils\dfLoading.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = df.groupby(['animal', 'task']).sess_block.cumsum()
c:\Users\dlab\rishika_sim\utils\dfLoading.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

loaded data


In [50]:
block_train_data['Emolga'].port.to_numpy().reshape(-1, 100).shape

(384, 100)